In [1]:
#pip install -qU langchain-core langchain-mistralai

In [1]:
import os
from pydantic import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_openai import ChatOpenAI
#from langchain_experimental.llms.ollama_functions import OllamaFunctions
from ollama_functions import OllamaFunctions
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [35]:
llm = ChatOllama(model="tomasonjo/llama3-text2cypher-demo")

In [2]:
llm = ChatOllama(model="llama3")

In [2]:
llm = OllamaFunctions(model="llama3")

In [32]:
os.environ["OPENAI_API_KEY"] = "sk-"

In [33]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [4]:
diffbot_api_key = "4c62797e105b35acb1a22d029adfd058"
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

In [3]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "pleaseletmein"

graph = Neo4jGraph()

In [6]:
model_name = "sentence-transformers/all-mpnet-base-v2"
# model_name = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\linew\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [4]:
graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

#entity_chain = prompt | llm.with_structured_output(Entities)
#entity_chain = llm.with_structured_output(Entities)
entity_chain = prompt | llm.with_structured_output(Entities)

In [5]:
entity_chain.invoke({"question": "Who is Harry Potter?"})

AttributeError: 'NoneType' object has no attribute 'create'

In [17]:
graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines 
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [38]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.content:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [39]:
structured_retriever("Who is Harry Potter?")

IndexError: list index out of range

In [34]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [32]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOllama(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [33]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

NameError: name 'retriever' is not defined

In [ ]:
chain.invoke({"question": "Which house did Harry potter belong to?"})

### Sepreate

In [7]:
#llm = ChatOllama(model="tomasonjo/llama3-text2cypher-demo")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question, convert it to a Cypher query. No pre-amble.",
        ),
        (
            "human",
            (
                "Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question: "
                "\n{schema} \nQuestion: {question} \nCypher query:"
            ),
        ),
    ]
)


In [8]:
chain = prompt | llm

In [10]:
question = "Who are the children of Harry Potter"

In [14]:
response = chain.invoke({"question": question, "schema": graph.schema})
print(response.content)

MATCH (harryPotter:Person {name: "Harry Potter"})-[:HAS_CHILD]->(child:Person)
RETURN child.name AS ChildName


In [ ]:
from neo4j import GraphDatabase

In [45]:
def read_query(query, params={}):
    with driver.session(database="neo4j") as session:
        results = session.execute_read(lambda tx: tx.run(query, params).data())       
        response = [list(record.values())[0] for record in results]
        return response

In [46]:
# DB URI and authentication
URI = "bolt://localhost:7687"
AUTH = ("neo4j","pleaseletmein")

In [47]:
driver = GraphDatabase.driver(URI, auth=AUTH)

In [5]:
response = read_query(response.content)
response

NameError: name 'read_query' is not defined

In [12]:
response = chain.invoke({"question": question, "schema": graph.schema})
response.content

'To answer this question, we would need to identify who Harry Potter is in our Neo4j graph. Since we have multiple people with the same name (e.g., Harry Potter), we\'ll use a unique identifier or property to pinpoint the correct person.\n\nLet\'s assume we have a property `name` on the `Person` node and Harry Potter has a unique name like "Severus Snape". We can then use this query:\n\n```cypher\nMATCH (p:Person {name: \'Severus Snape\'})-[:HAS_CHILD]->(c:Person)\nRETURN c.name AS childName\n```\n\nThis query starts by matching the `Person` node with the name "Severus Snape" (or any other unique identifier you have for Harry Potter). Then, it follows the `HAS_CHILD` relationship to find all the children of this person. Finally, it returns the names of these children.\n\nIf there are multiple children, this query will return a list of child names. If you want to retrieve more information about each child (e.g., their date of birth or cause of death), you can add additional patterns to 

In [ ]:
###
# def stream_text()
# for word in text.split(" "):
#yield word + " "
#time.sleep(0.5)

#st.write_stream(stream_text(str(answer)))